<a href="https://colab.research.google.com/github/apd1995/Stats-203V/blob/branch_1/Stats_203V_Special_Topics_Quantile_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are notes created by Apratim Dey while he was an instructor of the course Stats 203V on Linear Models and Regression Analysis in Summer, 2023, at Stanford University. If these notes are helpful to you, please cite them appropriately. For any feedback, please open an issue on GitHub.



# Quantile Regression



Let us start with the Wage dataset available in the ISLR package in R. Run the following code chunk to get the dataset up on your working environment and get a sneak peek into the data.

In [ ]:
install.packages('ISLR')
library(ISLR)
data("Wage")

# Preview the dataset
head(Wage)

The dataset and variable names are self explanatory. As you can probably guess by the name of the dataset, the goal will be to predict $\verb|wage|$ (column 11) based on the other variables. And then one can ask relevant questions, such as, whether age has a significant impact on salary, or whether having a college degree really helps.

But first, let's look at the variable $\verb|wage|$.


In [ ]:
wage = Wage$wage

# Plot histogram
hist(wage, breaks = 50, xlab = "wage", main = "Histogram of wage")

We notice something interesting (and typical about wages). While the major chunk of people in the dataset seem to be earning wages below 150, there are a few (rich!) people earning above 250.

Let's do a density plot of the data, which will clarify this further.

In [ ]:
# Plot density of wage
plot(density(wage))

The $\verb|wage|$ values have a long tail, and we are looking at a real example of positively skewed data! The following boxplot also certifies this.

In [ ]:
# Box and whisker plot of wage
boxplot(wage, pch = 19)

Let's see what fraction and number of people earn more than 250.

In [ ]:
rich_frac = mean(wage > 250)
rich_count = sum(wage > 250)
c(round(rich_frac, 3), rich_count)

So, 79 people (about 2.6% of the sample) earn significantly more than the others in the dataset.

All these statistics tell us that the rich people constitute a sizeable minority, distinct from the rest of the people. This class is somewhat rare, and perhaps has its own characteristics.

Anyway, we would like to predict wages based on the other variables. But first, let us remove the variables $\verb|logwage|$ (as it is simply the log of the wage), $\verb|region|$ (as it is the same for everyone in the dataset) and $\verb|year|$.

In [ ]:
# Let's rename the dataset to Wage_new after dropping the three columns
Wage_new = subset(Wage, select = -c(year, region, logwage))
head(Wage_new)

Notice that all the predictors (except age) are categorical. Let's build an ordinary linear regression model to predict wage now.

In [ ]:
# Performing linear regression
reg_ols = lm(wage ~ ., data = Wage_new)
summary(reg_ols)

Let's see how well the model has been able to fit the "rich" vs. the "bulk" class.

In [ ]:
# Collect the indices corresponding to the "rich" class
rich_indices = which(Wage_new$wage > 250)

# Residuals on the rich and bulk classes
residuals_ols = reg_ols$residuals
residuals_ols_rich = residuals_ols[rich_indices]
residuals_ols_bulk = residuals_ols[-rich_indices]

# Mean wage predicted on the rich class
predicted_ols_rich_mean = mean(reg_ols$fitted.values[rich_indices])

# Output as data frame
data.frame(RMSE_bulk = sqrt(mean(residuals_ols_bulk^2)),
RMSE_rich = sqrt(mean(residuals_ols_rich^2)),
predicted_rich_mean = predicted_ols_rich_mean,
actual_rich_mean = mean(wage[rich_indices]))

Notice how drastically worse the model has performed on the rich class.

This is not unexpected; this is precisely due to the rarity of the rich class. The model we built tries to fit the "bulk" or the majority. It does not do well for the rich group which constitutes a higher quantile of $\verb|wage|$. If we want to understand the rich community better, this general model  will not be good enough.

So, what do we do? Do we simply ignore the rest of the data and fit a model restricted only to the rich class? This approach would suffer from a higher variance as it wouldn't be able to leverage shared information across the full population.

Here is where quantile regression comes in. It aims to fit the higher quantiles of the dataset (and not just the mean, which ordinary linear models do). As a result, it takes into account the full dataset.

Let's first see the performance of quantile regression on this dataset. It can be implemented using the $\verb|rq|$ function in $\verb|quantreg|$ package in R.

In [ ]:
install.packages("quantreg")
library(quantreg)

# Fit quantile regression to 97.4th quantile (top 2.6%)
reg_quant = rq(wage ~ ., tau = 0.974, data = Wage_new)
reg_quant

We can simultaneously fit quantile regression to a grid of chosen quantiles without any extra effort. We simply add those quantiles as an array.

In [ ]:
# Fit quantile regression to the 50th, 97.4th, 98th and 99th percentiles
rq(wage ~ ., data = Wage_new, tau = c(0.5, 0.974, 0.98, 0.99))

Observe how the top quantiles are different from the median, say. For the median person, an advanced degree seems to provide much less benefit than what it provides for the top rich. The 99th percentile wages seem to be adversely affected if the person is widowed, but the lower percentiles are positively affected in the same case.

Now let us look at the coefficient estimates for the ordinary regression and quantile regression for the median.

In [ ]:
reg_quant_median = rq(wage ~ ., tau = 0.5, data = Wage_new)

data.frame(quantile_regression_coefs = reg_quant_median$coefficients,
ols_coefs = reg_ols$coefficients)

The estimates are not too different, suggesting that indeed the ordinary least squares regression was explaining the "average" or "median" individual in our dataset.

Now, let's see how well the quantile regression model built on the 98th percentile performs in predicting the top quantiles.

In [ ]:
# Perform quantile regression on 98th percentile
reg_quant_top = rq(wage ~ ., tau = 0.98, data = Wage_new)

# Get the residuals on the "rich" people and compute RMSE
residuals_quant_rich = reg_quant_top$residuals[rich_indices]
RMSE_quant_rich = sqrt(mean(residuals_quant_rich^2))

# Output data frame to compare with old RMSE from OLS
data.frame(RMSE_quant_rich = RMSE_quant_rich, RMSE_ols_rich = sqrt(mean(residuals_ols_rich^2)))

Observe how the RMSE has gone down significantly. We can also compare the average prediction.

In [ ]:
data.frame(predicted_quant_rich_mean = mean(reg_quant_top$fitted.values[rich_indices]),
predicted_old_rich_mean = predicted_ols_rich_mean,
true_old_rich_mean = mean(wage[rich_indices]))

Notice the improvement here as well.

It is important to remember though that the results of quantile regression will typically depend on the quantiles/ percentiles we choose to fit. If we are interested in modeling the 98th percentile, we would use the corresponding quantile regression, which may not give a good answer at another percentile, say, 50%.

The whole idea behind quantile regression is that one single global regression model describing the mean behavior of data may not be enough in many applications. A better idea is to model instead the various quantiles. Different quantiles may depend differently on the predictors, and this stratification provides useful information in decision making process.

## Technical words...

Now that we have talked about quantile regression through a data example, let's say a word or two about what it is doing, mathematically.

Ordinary linear regression minimizes the sum of squared errors: $\sum_{i=1}^n (y_i - x_i'\beta)^2$. The loss involved is the squared error loss.

Quantile regression uses a different loss function - the *check* loss. One can show that minimizing the check loss for a given percentile gives the corresponding quantile.